In [ ]:
!unzip /content/RAVDESS_dataset -d /content/RAVDESS_dataset/
# RAVDESS Dataset
# https://drive.google.com/file/d/1wWsrN2Ep7x6lWqOXfr4rpKGYrJhWc8z7/view

In [7]:
# OS BASED PATH
# base = Path(__file__).resolve().parent
# Notebook Path
import os
import glob

# All files and directories ending with .txt and that don't begin with a dot:
ROOT = os.path.abspath('')
dataset_path = os.path.join(ROOT, 'RAVDESS_dataset')

dataset_files = glob.glob(f"{dataset_path}/Actor_*/*.wav")

In [8]:
import librosa
import soundfile
import os, glob, pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

In [9]:
# Extract features (mfcc, chroma, mel) from a sound file
def extract_feature(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate = sound_file.samplerate
        if chroma:
            stft = np.abs(librosa.stft(X))
        result = np.array([])
        if mfcc:
            mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result = np.hstack((result, mfccs))
        if chroma:
            chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T, axis=0)
            result = np.hstack((result, chroma))
        if mel:
            mel = np.mean(librosa.feature.melspectrogram(y=X, sr=sample_rate).T, axis=0)
            result = np.hstack((result, mel))
    return result


In [10]:
# Emotions in the RAVDESS dataset
emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}

#Emotions to observe
focused_emotions=['neutral', 'calm', 'happy', 'sad', 'disgust', 'angry']

In [11]:
#Load the data and extract features for each sound file
def load_data(test_size=0.2):
    x,y=[],[]
    for file in dataset_files:
        file_name=os.path.basename(file)
        emotion=emotions[file_name.split("-")[2]]
        if emotion not in focused_emotions:
            continue
        feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append(emotion)
    return train_test_split(np.array(x), y, test_size=test_size, random_state=9)

In [12]:
#Split the dataset
x_train,x_test,y_train,y_test=load_data(test_size=0.25)

In [ ]:
x_train

In [13]:
#Get the shape of the training and testing datasets
print((x_train.shape[0], x_test.shape[0]))

(792, 264)


In [14]:
#Get the number of features extracted
print(f'Features extracted: {x_train.shape[1]}')

Features extracted: 180


In [15]:
#Initialize the Multi Layer Perceptron Classifier
model=MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(300,), learning_rate='adaptive', max_iter=500)

In [16]:
#Train the model
model.fit(x_train,y_train)

MLPClassifier(alpha=0.01, batch_size=256, hidden_layer_sizes=(300,),
              learning_rate='adaptive', max_iter=500)

In [17]:
#Predict for the test set
y_pred=model.predict(x_test)

In [18]:
y_pred

array(['happy', 'happy', 'disgust', 'sad', 'happy', 'happy', 'sad',
       'disgust', 'disgust', 'sad', 'sad', 'sad', 'happy', 'disgust',
       'angry', 'sad', 'happy', 'angry', 'disgust', 'sad', 'disgust',
       'happy', 'sad', 'disgust', 'sad', 'disgust', 'calm', 'happy',
       'sad', 'happy', 'angry', 'disgust', 'disgust', 'disgust', 'happy',
       'disgust', 'angry', 'disgust', 'angry', 'calm', 'calm', 'disgust',
       'disgust', 'sad', 'calm', 'happy', 'angry', 'sad', 'sad', 'sad',
       'calm', 'disgust', 'disgust', 'disgust', 'happy', 'sad', 'sad',
       'happy', 'sad', 'disgust', 'happy', 'happy', 'sad', 'happy',
       'happy', 'happy', 'sad', 'happy', 'happy', 'disgust', 'angry',
       'sad', 'disgust', 'disgust', 'angry', 'angry', 'sad', 'happy',
       'angry', 'happy', 'disgust', 'sad', 'calm', 'disgust', 'angry',
       'sad', 'neutral', 'happy', 'calm', 'calm', 'sad', 'sad', 'sad',
       'sad', 'happy', 'sad', 'happy', 'disgust', 'happy', 'sad', 'sad',
       'a

In [20]:
#Calculate the accuracy of our model
accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)

#Print the accuracy
print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 43.18%


In [21]:
from sklearn.metrics import accuracy_score, f1_score

In [22]:
f1_score(y_test, y_pred,average=None)

array([0.69135802, 0.28985507, 0.45378151, 0.53763441, 0.        ,
       0.36090226])

In [23]:
import pandas as pd
df=pd.DataFrame({'Actual': y_test, 'Predicted':y_pred})
df.head(20)

,Actual,Predicted
0,happy,happy
1,happy,happy
2,angry,disgust
3,disgust,sad
4,neutral,happy
5,angry,happy
6,sad,sad
7,calm,disgust
8,disgust,disgust
9,calm,sad


In [24]:
# import pickle
# # Writing different model files to file
# with open( 'modelForPrediction1.sav', 'wb') as f:
#     pickle.dump(model,f)
import joblib

clf_file = 'emotion_classifier_v2.pkl'

joblib.dump(model, clf_file)

['emotion_classifier_v2.pkl']

In [25]:
filename = 'emotion_classifier_v2.pkl'
loaded_model = joblib.load(open(filename, 'rb')) # loading the model file from the storage

feature=extract_feature(f"{dataset_path}/Actor_01/03-01-01-01-01-01-01.wav", mfcc=True, chroma=True, mel=True)

feature=feature.reshape(1,-1)

prediction=loaded_model.predict(feature)
prediction

array(['sad'], dtype='<U7')

In [26]:
feature

array([[-6.93497009e+02,  5.00643883e+01,  5.71450472e-01,
         1.43299656e+01,  3.33637023e+00, -2.54071975e+00,
        -4.05790901e+00, -1.07119989e+01, -7.29413891e+00,
         1.74018884e+00, -4.19064283e+00,  1.95466173e+00,
        -5.24789333e+00,  2.78142977e+00, -3.16756773e+00,
        -3.40008307e+00, -2.37803221e+00, -5.68717480e-01,
        -6.47753334e+00, -1.24320579e+00, -2.80542541e+00,
        -5.43635845e+00, -4.46875453e-01, -3.63516617e+00,
        -2.98372650e+00, -5.63902617e-01, -1.65101981e+00,
        -5.55944800e-01, -3.41018438e+00, -2.24465466e+00,
        -3.13058877e+00, -2.70089960e+00, -1.88821268e+00,
        -5.54154396e-01, -3.96459913e+00, -2.13485193e+00,
        -3.94577050e+00, -1.62457860e+00, -2.03990722e+00,
        -3.62910819e+00,  6.33734703e-01,  6.48760676e-01,
         6.21744275e-01,  6.34553194e-01,  6.60345674e-01,
         6.60464048e-01,  7.00949013e-01,  7.31556356e-01,
         7.46921480e-01,  7.23420441e-01,  7.16119647e-0